# Spotify Mood Session

## 1. Algorithm

a. **First Selection**. User selects a song $s_0$ from the set of all songs $S$ using some kind of search method (by name / artist) and plays that song.

b. **Played List**.  Let $P$ be a new list of played songs and let $P \gets s_0$   

c. **Cold Start**. Create a bootstrap random selection of $n$ songs from $S$ called $R$. Calculate the distance between each song and the first selection and select the closest to the chosen song . Formally:

$$
\forall s \in R \text{ dist}(s_i, s_0)
$$

*potential parameter: use a value to add the closest n songs to the initial bootstrap played list*

d. **Rating**. For every song that a user plays for more than 30 seconds, the song is rated $\text{pos}$, songs that are skipped within this time period are rated $\text{neg}$.  Songs that have not been played are rated $\text{non}$

e. **Recommending**.  Take the current set of vectors and select the next song based on some sort of agregated mean or nearest neighbour system.

repeat (d) and (e) ad infinitum.

Formally:

$$
\text{let: } s_0 \gets \text{ user}(S) \\
\text{let: } P = \{ s_0 \} \\
\text{let: } R \subset S \\
\text{let: } D = \emptyset \\ 
\text{for } s_i \text{ in } R:\\
D_i = \text{dist}(s_i, s_0) \\
\text{end for }\\
\text{select}(\text{max}(D))\\
$$

In [1]:
import json
import pandas as pd
import os
from IPython.display import display

# create list of playlists
playlists = []
path = 'data'
count = 5
for filename in os.listdir(path):
    count -= 1
    if count < 0:
        break
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        d = json.load(open(f))
        playlists.append(pd.DataFrame(d['playlists']))

playlists = pd.concat(playlists)
playlists = playlists.reset_index(drop=True)
print(playlists.shape)

(5000, 12)


In [2]:
# Add all songs from the playlists into a single list of songs
songs = [] 
for i in range(len(playlists)):
    tracks = playlists.iloc[i].loc['tracks']
    for track in tracks:
        songs.append(track)

print(len(songs))
print(songs[0])

334487
{'pos': 0, 'artist_name': 'Missy Elliott', 'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI', 'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk', 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)', 'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K', 'duration_ms': 226863, 'album_name': 'The Cookbook'}


In [4]:
songs_df = pd.DataFrame(songs)
print(songs_df.shape)
songs_df.head()

(334487, 8)


,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot


In [30]:
# Remove duplicate songs by URI
songs_df.drop_duplicates('track_name')

,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,Lost Frequencies,spotify:track:2vCtiBvJJZfz773yTfAxPP,spotify:artist:7f5Zgnp2spUuuzKplmRkt7,What Is Love 2016 - Mike Mago Remix,spotify:album:4N1e8k2o2NN932Y8xgZV1p,209882,What Is Love 2016
1,1,John Legend,spotify:track:5WOLZP8KrXiupBjG1SSN5U,spotify:artist:5y2Xq6xcjJb2jVM54GHK3t,Love Me Now - Dave Audé Remix Radio Edit,spotify:album:2CFZOjwtV7B6XzULEwIdHN,219106,Love Me Now (Remixes)
2,2,July Child,spotify:track:0oQDQ9QiqsO63EEBAro8Le,spotify:artist:3Jghk94Gog2dhBAO7ZZaM8,Thinkin of U,spotify:album:2i0oT2HtRN4o3P45zQjel7,194000,Thinkin of U
3,3,Tiësto,spotify:track:7MUS0La2IQ85vJ59fQqtoN,spotify:artist:2o5jDhtHVPhrJdv3cEQ99Z,I Will Be Here - (Wolfgang Gartner Remix),spotify:album:2qY9UxNEbQR2O2ECc3Fd48,237066,Kaleidoscope Remixed
4,4,Joe Stone,spotify:track:3ciyZYofjiqmMUElM5qgGB,spotify:artist:4kwEd1P9j15ZqUVP5zK7Pv,The Party (This Is How We Do It),spotify:album:3SBDe6WBbEXNtjraPxXLOh,185807,The Party (This Is How We Do It)
...,...,...,...,...,...,...,...,...
334452,9,Skylar Grey,spotify:track:2SbF8UR7YvY3ArOH8MqgrC,spotify:artist:4utLUGcTvOJFr6aqIJtYWV,Tower (Don't Look Down),spotify:album:0xpiZPRZ2nzVgINpDHaW4Y,240560,Don't Look Down
334455,12,Ed Sheeran,spotify:track:24ha18yq0t7XnZ9EgJ7HYb,spotify:artist:6eUKZXaKkcviH0Ku9w2n3V,So,spotify:album:2hyDesSAYNefikDJXlqhPE,268760,5
334457,14,Rusty Clanton,spotify:track:7jxYkSGQdhdDD940NdxiSv,spotify:artist:1llFhhYvVi6kC2bfKoPw2k,The Very Thought of You,spotify:album:6Udmc2m8vKv2Vpqts2iEUV,120372,The Very Thought of You
334463,20,gnash,spotify:track:50J9NhjECm19ZBadwJembA,spotify:artist:3iri9nBFs9e4wN7PLIetAw,fuck me up,spotify:album:65OO1Lvu27sSpND6QycjcW,174000,me
